# Alphavec example

In this example we'll walkthrough creating and backtesting a simple cost-aware crypto trend strategy.

In [ ]:
import os
import sys
from pathlib import PurePath
from functools import partial
from IPython.display import display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import alphavec.backtest as vbt

workspace_root = str(PurePath(os.getcwd()))
sys.path.append(workspace_root)

plt.rc("figure", figsize=(16, 6))
plt.rc("savefig", dpi=90)
plt.rc("font", family="sans-serif")
plt.rc("font", size=14)

%matplotlib inline
%reload_ext autoreload
%autoreload 3

Let's begin by loading our data.

We transform a flat file of candlestick (kline) data for a variety of crypto assets into the required format for backtesting: a single level DatetimeIndex with a column for each asset.

In [ ]:
def ohlcv_from_csv(filename):
    return pd.read_csv(
        filename,
        index_col=["symbol", "dt"],
        parse_dates=["dt"],
        dtype={
            "o": np.float64,
            "h": np.float64,
            "l": np.float64,
            "c": np.float64,
            "v": np.float64,
        },
        dayfirst=True,
    )


prices_filename = f"{workspace_root}/tests/testdata/binance-margin-1d.csv"
market = ohlcv_from_csv(prices_filename)
market = market[~market.index.duplicated()]
market = market.unstack(level=0).sort_index(axis=1).stack()

# Close price series for calulating our strategy weights.
strategy_prices = pd.DataFrame(
    market.loc[:, ["c"]].unstack(level=1).droplevel(level=0, axis=1)
)

# Open price series, used to calculate open-to-open returns.
# When returns are shifted -2 periods during the backtest, implies we can execute our trades
# at the open price of the next day / bar.
trade_prices = pd.DataFrame(
    market.loc[:, ["o"]].unstack(level=1).droplevel(level=0, axis=1)
)

display(trade_prices.tail())

Using the prices we create a simple risk-adjusted moving average cross forecast.

In [ ]:
lookback = 16
forecast = (
    strategy_prices.ewm(span=lookback, adjust=False).mean()
    - strategy_prices.ewm(span=lookback * 4, adjust=False).mean()
).div(strategy_prices.diff().ewm(span=20).std())

forecast.plot()

Next we scale the forecast to acheive an absolute median value of 1. 

Note we use a rolling method to prevent look-ahead bias.

In [ ]:
def roll_scale(x: pd.DataFrame, scaler, window=252, min_periods=20) -> pd.DataFrame:
    return x.rolling(window=window, min_periods=min_periods).apply(
        lambda x: scaler(x)[-1], raw=True
    )


def absmedian_scale(x):
    absavg = np.median(np.abs(x))
    scalar = 1 if absavg == 0 else 1 / absavg
    return x * scalar


forecast = forecast.pipe(roll_scale, scaler=absmedian_scale)
forecast.plot()

All asset forecasts now share a common scale, enabling us to apply a function to attenuate large forecasts and guard against reversion at the extremes of a trend.

In [ ]:
def reverting_sigmoid(x):
    return x * np.exp(-(x**2))


forecast = forecast.map(reverting_sigmoid)
forecast.plot()

The final strategy weights are formed by normalizing the portfolio allocations so they sum to 1 at each interval.

In [ ]:
weights = forecast.div(forecast.abs().sum(axis=1), axis=0)
display(weights.tail())
display(weights.describe())
weights.plot()

Finally we are ready to backtest.

However, we must take care to align the prices and weights.

During the backtest the returns calculated from the prices will be _shifted_ to prevent look-ahead bias.

Note this is a cost-aware backtest with fixed leverage. We've chosen cost parameters that mimic a typical crypto exchange.

In [ ]:
weights *= 2  # Apply fixed 2x leverage

weights = weights["2019-01-01":]
trade_prices = trade_prices.mask(weights.isna())
trade_prices, weights = trade_prices.align(weights, join="inner")

perf, perf_pnl, perf_sr, port_perf, port_rets = vbt.backtest(
    weights,
    trade_prices,
    freq_day=1,
    trading_days_year=365,
    shift_periods=2,
    commission_func=partial(vbt.pct_commission, fee=0.001),
    ann_borrow_rate=0.05,
    spread_pct=0.0005,
    ann_risk_free_rate=0,
    bootstrap_n=1000,
)

Let's look at the performance of the strategy, starting with the asset-wise view, then the portfolio view.

Note Sharpe and volatility are annualized measures.

In [ ]:
display(perf)
perf_pnl.plot()
perf_sr.plot()

In [ ]:
display(port_perf)

In [ ]:
port_rets.resample("Y").sum().rename(lambda x: x.strftime("%Y")).plot(kind="bar")

In [ ]:
perf_sr.loc[:, ("portfolio", "SR")].resample("Y").last().rename(
    lambda x: x.strftime("%Y")
).plot(kind="bar")

Simulate 1000 USD initial investment

In [ ]:
(1000 * vbt.pnl(port_rets)).plot()

In [ ]:
port_cagr = port_perf.loc["observed", "cagr"]
bench_cagr = (
    perf.loc["BTCUSDT", ("asset", "cagr")] * 0.5
    + perf.loc["ETHUSDT", ("asset", "cagr")] * 0.5
)

info_ratio = (port_cagr - bench_cagr) / port_perf.loc["observed", "annual_volatility"]

info_ratio

In [ ]:
# TODO: calc alpha and beta

In [ ]:
port_rets.hist(bins=100)
plt.show()
display(port_rets.skew(), port_rets.kurtosis())

## Next Steps

Check out our upcoming Alphaone project <https://github.com/breaded-xyz/alphaone> for a real-world strategy that has been validated with Alphavec and is running in live.